In [1]:
import sympy as sm

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex="mathjax", pretty_print=False)

In [41]:
from sympy.physics.mechanics import dynamicsymbols

In [203]:
m1, m2, l1, l2, g = sm.symbols('m1, m2, l1, l2, g', real=True)
theta1, theta2 = dynamicsymbols('theta1, theta2')
theta1d = dynamicsymbols('theta1',1)
theta2d = dynamicsymbols('theta2',1)
theta1dd = dynamicsymbols('theta1',2)
theta2dd = dynamicsymbols('theta2',2)
g, m1, m2, l1, l2, theta1, theta2, theta1d, theta2d, theta1dd, theta2dd

(g, m1, m2, l1, l2, theta1, theta2, theta1', theta2', theta1'', theta2'')

In [204]:
x1 = l1*sm.sin(theta1)
y1 = l1*sm.cos(theta1)
x2 = l1*sm.sin(theta1) + l2*sm.sin(theta1 + theta2)
y2 = l1*sm.cos(theta1) + l2*sm.cos(theta1 + theta2)

x1_dot = sm.diff(x1,'t')
x2_dot = sm.diff(x2,'t')
y1_dot = sm.diff(y1,'t')
y2_dot = sm.diff(y2,'t')

In [205]:
# kinetic energy
ke = (1/2)*m1*sm.Pow(x1_dot,2)
ke += (1/2)*m1*sm.Pow(y1_dot,2)
ke += (1/2)*m2*sm.Pow(x2_dot,2)
ke += (1/2)*m2*sm.Pow(y2_dot,2)
ke

0.5*l1**2*m1*sin(theta1)**2*theta1'**2 + 0.5*l1**2*m1*cos(theta1)**2*theta1'**2 + 0.5*m2*(-l1*sin(theta1)*theta1' - l2*(theta1' + theta2')*sin(theta1 + theta2))**2 + 0.5*m2*(l1*cos(theta1)*theta1' + l2*(theta1' + theta2')*cos(theta1 + theta2))**2

In [206]:
# potential energy
pe = m1*g*y1
pe += m2*g*y2
pe

g*l1*m1*cos(theta1) + g*m2*(l1*cos(theta1) + l2*cos(theta1 + theta2))

In [207]:
# the lagrangian equation
L = ke - pe
L

-g*l1*m1*cos(theta1) - g*m2*(l1*cos(theta1) + l2*cos(theta1 + theta2)) + 0.5*l1**2*m1*sin(theta1)**2*theta1'**2 + 0.5*l1**2*m1*cos(theta1)**2*theta1'**2 + 0.5*m2*(-l1*sin(theta1)*theta1' - l2*(theta1' + theta2')*sin(theta1 + theta2))**2 + 0.5*m2*(l1*cos(theta1)*theta1' + l2*(theta1' + theta2')*cos(theta1 + theta2))**2

In [208]:
# force applied at joint 1
f1 = sm.diff(sm.diff(L,theta1d),'t') - sm.diff(L,theta1)
f1sim = sm.expand(sm.simplify(f1))
f1sim

-1.0*g*l1*m1*sin(theta1) - 1.0*g*l1*m2*sin(theta1) - 1.0*g*l2*m2*sin(theta1 + theta2) + 1.0*l1**2*m1*theta1'' + 1.0*l1**2*m2*theta1'' - 2.0*l1*l2*m2*sin(theta2)*theta1'*theta2' - 1.0*l1*l2*m2*sin(theta2)*theta2'**2 + 2.0*l1*l2*m2*cos(theta2)*theta1'' + 1.0*l1*l2*m2*cos(theta2)*theta2'' + 1.0*l2**2*m2*theta1'' + 1.0*l2**2*m2*theta2''

In [209]:
# force applied at joint 2
f2 = sm.diff(sm.diff(L,theta2d),'t') - sm.diff(L,theta2)
f2sim = sm.expand(sm.simplify(f2))
f2sim

-1.0*g*l2*m2*sin(theta1 + theta2) + 1.0*l1*l2*m2*sin(theta2)*theta1'**2 + 1.0*l1*l2*m2*cos(theta2)*theta1'' + 1.0*l2**2*m2*theta1'' + 1.0*l2**2*m2*theta2''

In [210]:
#inertia term
B11 = f1sim.coeff(theta1dd) * theta1dd
B12 = f2sim.coeff(theta1dd) * theta1dd

B21 = f1sim.coeff(theta2dd) * theta2dd
B22 = f2sim.coeff(theta2dd) * theta2dd

B = sm.Matrix([[B11, B21], [B12, B22]])
B

Matrix([
[(1.0*l1**2*m1 + 1.0*l1**2*m2 + 2.0*l1*l2*m2*cos(theta2) + 1.0*l2**2*m2)*theta1'', (1.0*l1*l2*m2*cos(theta2) + 1.0*l2**2*m2)*theta2''],
[                              (1.0*l1*l2*m2*cos(theta2) + 1.0*l2**2*m2)*theta1'',                              1.0*l2**2*m2*theta2'']])

In [211]:
#centrifugal / Coriolis force term,
C11 = (f1sim.coeff(theta1d * theta2d)*theta1d * theta2d) + (f1sim.coeff(theta1d * theta1d)* theta1d * theta1d) + (f1sim.coeff(theta2d * theta2d)*theta2d * theta2d)
C12 = (f2sim.coeff(theta1d * theta2d)*theta1d * theta2d) + (f2sim.coeff(theta1d * theta1d)* theta1d * theta1d) + (f2sim.coeff(theta2d * theta2d)*theta2d * theta2d)

C = sm.Matrix([C11, C12])
C

Matrix([
[-2.0*l1*l2*m2*sin(theta2)*theta1'*theta2' - 1.0*l1*l2*m2*sin(theta2)*theta2'**2],
[                                            1.0*l1*l2*m2*sin(theta2)*theta1'**2]])

In [212]:
# gravity term
g1 = f1sim - sm.expand(B11) - sm.expand(B21) - sm.expand(C11)
g2 = f2sim - sm.expand(B12) - sm.expand(B22) - sm.expand(C12)
g = sm.Matrix([g1, g2])
g

Matrix([
[-1.0*g*l1*m1*sin(theta1) - 1.0*g*l1*m2*sin(theta1) - 1.0*g*l2*m2*sin(theta1 + theta2)],
[                                                    -1.0*g*l2*m2*sin(theta1 + theta2)]])